SSL 证书导入

In [1]:
import pip_system_certs.wrapt_requests

建立 LLM 会话  
在报告页找免费 api_key 需要根据模型来源初始化会话

In [2]:
import json
with open(r".\model_mapper.json", "r", encoding="utf8") as reader:
    model_map = json.load(reader)
    model_avails = {mn:k for k in model_map.keys() for mn in model_map[k]["model_list"]}

print("可选模型 (选一个填到 model_name):")
for mak in model_avails.keys(): print(mak)

可选模型 (选一个填到 model_name):
gemini-2.5-flash
deepseek/deepseek-r1-0528-qwen3-8b:free
x-ai/grok-code-fast-1
anthropic/claude-3.5-haiku


In [ ]:
# 根据 json 中记录的可用模型修改
model_name = "gemini-2.5-flash"

In [4]:
assert model_name in model_avails, "check model_name"
frame = model_avails[model_name]
params = {k: v for k, v in model_map[frame].items() if k not in ["model_list"]}

if frame == "google":
    from google import genai
    client = genai.Client(**params)
elif frame == "openrouter":
    from openai import OpenAI
    client = OpenAI(**params)
else: raise NotImplementedError

认证 github token

In [5]:
from github.Repository import Repository, NotSet
from github.PullRequest import PullRequest

import github

with open(r".\github_token", "r", encoding="utf8") as reader:
    access_token = reader.readline()

auth = github.Auth.Token(access_token)

筛选 PR

In [6]:
from typing import Literal, List

def filtre(
        repo:Repository,
        pr_state:Literal["open", "close", "all"]=NotSet, # pr 状态
        sort:Literal["created", "updated", "pushed", "popularity"]=NotSet, # 结果排序方式
        direction:Literal["asc", "desc"]=NotSet, # 结果顺序
        base:str=NotSet, # 目标分支
        head:str=NotSet, # 来源分支/来源用户
        prid:int=None, # 指定 pr 编号
        author:str=None, # 指定 pr 发起人
        day_back:int=None, # 回溯指定天数内
    ) -> List[PullRequest]:
    results = repo.get_pulls(pr_state, sort, direction, base, head)
    if prid: results = [pr for pr in results if pr.number == prid]
    if author: results = [pr for pr in results if pr.user.login == author]
    if day_back:
        # datetime 范围内
        from datetime import datetime, timedelta, timezone
        now = datetime.now().replace(tzinfo=timezone.utc)
        start_from = now - timedelta(days=day_back) 
        results = [pr for pr in results if pr.created_at >= start_from]
    return results

In [ ]:
## https://github.com/{browser-use/browser-use}/pull/xxx.diff
repository = "browser-use/browser-use"

with github.Github(auth=auth) as g:
    repo = g.get_repo(repository)
    prs = filtre(repo, "open", day_back=2) ## 假设在内部，则用 author + day_back 可以查到指定用户的 PR，这里随便选了一些。。
    if not prs: raise ValueError("No matching PR found")

prompt 模板 需要调整

In [8]:
# user_prompt = """Please briefly summarize the changes made to the code from the following diff:
# """
user_prompt = """请简短总结以下的 PR 描述和代码差异，并用数字编号输出差异条目：
PR 描述：
{desc}
代码差异：
{diffs}
"""

In [9]:
from utils import file_io_utils as fiu
from tqdm.notebook import tqdm

def get_llm_resp(query:str):
    if frame == "google":
        return client.models.generate_content(
            model=model_name,
            contents=[query]
        ).text
    elif frame == "openrouter":
        
        return client.chat.completions.create(
            model=model_name,
            messages=[{"role": "user", "content": query}],
            max_tokens=500
        ).choices[0].message.content
    else: raise NotImplementedError

def get_llm_diff_resp(pr):
    diffs = fiu.get_remote_contents(pr.diff_url)
    return get_llm_resp(user_prompt.format(desc=f"{pr.title}\n{pr.body}", diffs=diffs))
    
responses = [get_llm_diff_resp(pr) for pr in tqdm(prs)]

  0%|          | 0/5 [00:00<?, ?it/s]

汇总成简报

In [10]:
summary_prompt = """请根据以下代码差异描述，总结出一份简报，要求根据日期排序："""
for i, resp in enumerate(responses[::-1]): # 从旧到新
    summary_prompt += f"\n\n{prs[::-1][i].created_at}:\n\n{resp}\n"

In [11]:
response = get_llm_resp(summary_prompt)

In [12]:
from IPython.display import Markdown, display
display(Markdown(response))

好的，这是一份根据您提供的代码差异描述整理的简报，已按日期排序：

---

**代码差异简报**

以下是根据提供的代码差异描述整理的简报，按日期排序。

### 2025-09-11

*   **09:35:41+00:00：【新功能】引入 CaptchaWatchdog 机制，提升代理处理验证码的可靠性。**
    *   此更新通过在验证码识别期间暂停代理并在识别完成后恢复代理，显著提高了代理在受验证码保护网站上的稳定性。
    *   主要修改包括：在代理执行循环中添加验证码暂停状态检查；在 `Session` 类中增加 `agent_paused_for_captcha` 状态和 `_captcha_watchdog` 实例；新增 `CaptchaWatchdog` 文件，监听 `BrowserUse.captchaSolverStarted` 和 `BrowserUse.captchaSolverFinished` 事件以控制代理暂停/恢复；并更新了 `pyproject.toml`。

*   **15:35:38+00:00：【缺陷修复】修正 `on_ScrollToTextEvent` 中的 `UnboundLocalError`，确保 `js_result` 始终初始化，并改进文本查找逻辑。**
    *   解决了 `on_ScrollToTextEvent` 函数中 `js_result` 未初始化导致的 `UnboundLocalError`。
    *   修复措施包括：在函数开始时将 `js_result` 初始化为 `None`，并重构了文本查找成功后的判断和错误处理逻辑，以正确处理 XPath 查找和 JavaScript 回退两种情况，避免不必要的错误抛出。

*   **20:01:42+00:00：【新功能】新增 `hover_element` 操作，使代理能够悬停在元素上，增强现代 Web 应用的自动化能力。**
    *   引入了 `hover_element` 操作，允许代理通过元素索引进行悬停，从而触发工具提示、下拉菜单和浮层等交互。
    *   该功能利用 Chrome DevTools Protocol (CDP) 精确移动鼠标，并返回悬停坐标用于调试。主要涉及新增 `HoverElementAction` 模型、`HoverElementEvent` 事件、`hover_element` 公共动作注册以及 `on_HoverElementEvent` 异步事件处理器。

*   **21:47:53+00:00：【集成】为 Mistral 模型添加了一等支持，引入 `ChatMistral` 提供者，并优化了其参数兼容性与结构化输出处理。**
    *   新增 `ChatMistral` 提供者，继承自 `ChatOpenAI`，为 Mistral 模型提供了原生支持。
    *   `ChatMistral` 默认配置 Mistral 模型和基 URL，从 `MISTRAL_API_KEY` 获取密钥，并解决了 `max_completion_tokens` 到 `max_tokens` 的参数兼容性问题。同时，改进了结构化输出处理，包括客户端验证和对列表包裹 JSON 响应的容错处理。
    *   更新了文档并增加了可运行示例。

### 2025-09-12

*   **03:17:13+00:00：【文档更新】更新 `news-use` 示例的 README 文件，改进设置说明并强调版本要求。**
    *   此更新将 `browser-use v0.7.7+` 的版本要求提示从“CAUTION”更改为更重要的“IMPORTANT”。
    *   同时，在 `news-use` 示例的 README 中添加了克隆仓库和切换到应用文件夹的 `git clone` 和 `cd` 指令，以提升用户设置体验。

---

对自己的 pr 添加评论 模拟发布汇报

In [ ]:
with github.Github(auth=auth) as g:
    repo = g.get_repo("icm162/pgat") # 测试远程仓库 
    pr = repo.get_pull(1) # 第一个 PR
    pr.create_issue_comment(response) # 创建评论 网页支持 markdown